**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue May  3 08:05:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
my_local_drive='/content/gdrive/My\ Drive/TER/WhaleAttributesDetection/'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

/content/gdrive/My Drive/TER/WhaleAttributesDetection


'/content/gdrive/My Drive/TER/WhaleAttributesDetection'

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
# also set CUDNN, CUDNN_HALF and LIBSO to 1

%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make

/content/gdrive/My Drive/TER/WhaleAttributesDetection/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
   

### Make prediction

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-custom.cfg
%cd ..

/content/gdrive/MyDrive/TER/WhaleAttributesDetection/darknet/cfg
/content/gdrive/MyDrive/TER/WhaleAttributesDetection/darknet


In [ ]:
!cat cfg/yolov4-custom.cfg

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-testing.cfg training/yolov4-custom_best.weights /content/gdrive/My\ Drive/TER/data/test_img/wC_4862.jpg -thresh 0.3

### Bounding box pred for all images

In [ ]:
!unzip imgs.zip 

In [ ]:
!ls imgs/wC*.jpg | wc -l

In [ ]:
!wc -l imgs_path.txt 

In [ ]:
!cat yolov4-testing.cfg

In [ ]:
!./darknet detector test data/obj.data yolov4-testing.cfg training/yolov4-custom_best.weights -ext_output -out train.json < imgs_path.txt -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
../imgs/wC_3415.jpg: Predicted in 32.030000 milli-seconds.
head: 38%	(left_x:  127   top_y:   42   width:  136   height:  127)
air: 80%	(left_x:  426   top_y:  140   width:   72   height:   49)
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
../imgs/wC_2548.jpg: Predicted in 31.999000 milli-seconds.
air: 84%	(left_x:   94   top_y:  184   width:   80   height:   64)
head: 50%	(left_x:  280   top_y:   47   width:  134   height:  170)
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
../imgs/wC_9907.jpg: Predicted in 31.923000 milli-seconds.
head: 55%	(left_x:  230   top_y:  428   width:  292   height:  204)
air: 88%	(left_x:  234   top_y:   98   width:   96   height:   88)
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detec